Contains combination of checkpoint 1, 2, and 3 plus an additional experiment for Table 10.

Checkpoint for Table 10 is not available due to possible long runtime due to many permutations of experiment's variables.

Because of this, checkpoint for Table 10 is combined with checkpoint 1 (in this file) that also uses the same permutations.

Instead of modifying checkpoint 1 directly, I opted to create this unified reproduction pipeline that also encapsulates Table 10.

_Contains different naming convention & cell execution from previous checkpoints._

# Pre-experiment

## Machine Preparation

In [ ]:
%%bash
pwd
ls -a
du ~ -hs
date

In [ ]:
!sudo add-apt-repository ppa:deadsnakes/ppa -y > /dev/null 2>&1

In [ ]:
%%bash
sudo apt -qq update -y
sudo apt -qq upgrade -y

In [ ]:
!sudo DEBIAN_FRONTEND=noninteractive apt install python3.9 python3.9-venv unzip -yqq

In [ ]:
!mkdir artifact_checkgpt

In [ ]:
!python3.9 -m venv ./artifact_checkgpt/.venv

In [ ]:
%%bash
wget -nv https://zenodo.org/records/11215222/files/CheckGPT.zip?download=1 -O ./artifact_checkgpt/CheckGPT.zip
wget -nv https://zenodo.org/records/11215222/files/CheckGPT_presaved_files.zip?download=1 -O ./artifact_checkgpt/CheckGPT_presaved_files.zip
wget -nv https://zenodo.org/records/11215222/files/CS.zip?download=1 -O ./artifact_checkgpt/CS.zip
wget -nv https://zenodo.org/records/11215222/files/PHX.zip?download=1 -O ./artifact_checkgpt/PHX.zip
wget -nv https://zenodo.org/records/11215222/files/HSS.zip?download=1 -O ./artifact_checkgpt/HSS.zip
wget -nv https://zenodo.org/records/11215222/files/GPABenchmark.zip?download=1 -O ./artifact_checkgpt/GPABenchmark.zip
wget -nv https://zenodo.org/records/11215222/files/scripts.zip?download=1 -O ./artifact_checkgpt/scripts.zip
wget -nv https://zenodo.org/records/11215222/files/README.md?download=1 -O ./artifact_checkgpt/README.md
wget -nv https://zenodo.org/records/11215222/files/requirements.txt?download=1 -O ./artifact_checkgpt/requirements.txt

In [ ]:
%%bash
unzip -q ./artifact_checkgpt/CheckGPT.zip -d ./artifact_checkgpt/
unzip -q ./artifact_checkgpt/CheckGPT_presaved_files.zip -d ./artifact_checkgpt/
mkdir ./artifact_checkgpt/GPABench2/
unzip -q ./artifact_checkgpt/CS.zip -d ./artifact_checkgpt/GPABench2/
unzip -q ./artifact_checkgpt/PHX.zip -d ./artifact_checkgpt/GPABench2/
unzip -q ./artifact_checkgpt/HSS.zip -d ./artifact_checkgpt/GPABench2/
unzip -q ./artifact_checkgpt/GPABenchmark.zip -d ./artifact_checkgpt/
unzip -q ./artifact_checkgpt/scripts.zip -d ./artifact_checkgpt/CheckGPT/

In [ ]:
%%bash
rm ./artifact_checkgpt/CheckGPT.zip
rm ./artifact_checkgpt/CheckGPT_presaved_files.zip
rm ./artifact_checkgpt/CS.zip
rm ./artifact_checkgpt/PHX.zip
rm ./artifact_checkgpt/HSS.zip
rm ./artifact_checkgpt/GPABenchmark.zip
rm ./artifact_checkgpt/scripts.zip

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate

python --version

pip install -r ./artifact_checkgpt/requirements.txt -q
pip install datetime -q
python -m nltk.downloader punkt

## Source Code Modification

Adjustment to `dnn.py` needed to run experiment on CNN classification head.

In [ ]:
%%bash
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/
sed -i 's/if MODELID == 14:/if MODELID in [4]:/' dnn.py

Adjustment to `dnn.py` needed to run experiment on prompt transferability.

In [ ]:
%%bash
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

sed -i 's/domain_list, task_list, prompt_list = \[domain\], \[task\], \[prompt\]/domain_list, task_list, prompt_list = \[domain\], \[task\], \[int(d) for d in str(prompt)\]/' dnn.py

## Extra Logic

In [ ]:
import datetime

def now():
    time = datetime.datetime.now().timestamp()
    return time

def elapsed_time(start_time: float):
    HIS = 3600
    MIS = 60
    end_time = datetime.datetime.now().timestamp()
    elapsed_seconds = end_time - start_time
    elapsed_hours = int(elapsed_seconds / HIS)
    elapsed_seconds = elapsed_seconds - (elapsed_hours * HIS)
    elapsed_minutes = int(elapsed_seconds / MIS)
    elapsed_seconds = int(elapsed_seconds - (elapsed_minutes * MIS))
    elapsed_time = f"{elapsed_hours} hour(s) {elapsed_minutes} minute(s) {elapsed_seconds} second(s)"
    print(elapsed_time)

# Experiments

Reproduction's working directory will always be `~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/`.

In [ ]:
global_start = now()
start = now()

## CS - 5000

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python features.py CS 1 1 --gpt 0 --number 5000
python features.py CS 2 1 --gpt 0 --number 5000

python features.py CS 1 1 --gpt 1 --number 5000
python features.py CS 1 2 --gpt 1 --number 5000
python features.py CS 1 3 --gpt 1 --number 5000
python features.py CS 1 4 --gpt 1 --number 5000

python features.py CS 2 1 --gpt 1 --number 5000
python features.py CS 2 2 --gpt 1 --number 5000
python features.py CS 2 3 --gpt 1 --number 5000
python features.py CS 2 4 --gpt 1 --number 5000

python features.py CS 3 1 --gpt 1 --number 5000
python features.py CS 3 2 --gpt 1 --number 5000
python features.py CS 3 3 --gpt 1 --number 5000
python features.py CS 3 4 --gpt 1 --number 5000

In [ ]:
elapsed_time(start)
start = now()

### Table 7 & 8 - CheckGPT

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_5000 1 1 Train_CS_CheckGPTArch_Task1_Prompt1 --batchsize 256
python dnn.py CS_5000 1 2 Train_CS_CheckGPTArch_Task1_Prompt2 --batchsize 256
python dnn.py CS_5000 1 3 Train_CS_CheckGPTArch_Task1_Prompt3 --batchsize 256
python dnn.py CS_5000 1 4 Train_CS_CheckGPTArch_Task1_Prompt4 --batchsize 256

python dnn.py CS_5000 2 1 Train_CS_CheckGPTArch_Task2_Prompt1 --batchsize 256
python dnn.py CS_5000 2 2 Train_CS_CheckGPTArch_Task2_Prompt2 --batchsize 256
python dnn.py CS_5000 2 3 Train_CS_CheckGPTArch_Task2_Prompt3 --batchsize 256
python dnn.py CS_5000 2 4 Train_CS_CheckGPTArch_Task2_Prompt4 --batchsize 256

python dnn.py CS_5000 3 1 Train_CS_CheckGPTArch_Task3_Prompt1 --batchsize 256
python dnn.py CS_5000 3 2 Train_CS_CheckGPTArch_Task3_Prompt2 --batchsize 256
python dnn.py CS_5000 3 3 Train_CS_CheckGPTArch_Task3_Prompt3 --batchsize 256
python dnn.py CS_5000 3 4 Train_CS_CheckGPTArch_Task3_Prompt4 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_5000 1 1 Train_CS_CheckGPTArch_Task1_Prompt1_Test_CS_Task1_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CheckGPTArch_Task1_Prompt1/Best_CS_5000_Task1.pth
python dnn.py CS_5000 1 2 Train_CS_CheckGPTArch_Task1_Prompt2_Test_CS_Task1_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CheckGPTArch_Task1_Prompt2/Best_CS_5000_Task1.pth
python dnn.py CS_5000 1 3 Train_CS_CheckGPTArch_Task1_Prompt3_Test_CS_Task1_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CheckGPTArch_Task1_Prompt3/Best_CS_5000_Task1.pth
python dnn.py CS_5000 1 4 Train_CS_CheckGPTArch_Task1_Prompt4_Test_CS_Task1_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CheckGPTArch_Task1_Prompt4/Best_CS_5000_Task1.pth

python dnn.py CS_5000 2 1 Train_CS_CheckGPTArch_Task2_Prompt1_Test_CS_Task2_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CheckGPTArch_Task2_Prompt1/Best_CS_5000_Task2.pth
python dnn.py CS_5000 2 2 Train_CS_CheckGPTArch_Task2_Prompt2_Test_CS_Task2_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CheckGPTArch_Task2_Prompt2/Best_CS_5000_Task2.pth
python dnn.py CS_5000 2 3 Train_CS_CheckGPTArch_Task2_Prompt3_Test_CS_Task2_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CheckGPTArch_Task2_Prompt3/Best_CS_5000_Task2.pth
python dnn.py CS_5000 2 4 Train_CS_CheckGPTArch_Task2_Prompt4_Test_CS_Task2_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CheckGPTArch_Task2_Prompt4/Best_CS_5000_Task2.pth

python dnn.py CS_5000 3 1 Train_CS_CheckGPTArch_Task3_Prompt1_Test_CS_Task3_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CheckGPTArch_Task3_Prompt1/Best_CS_5000_Task3.pth
python dnn.py CS_5000 3 2 Train_CS_CheckGPTArch_Task3_Prompt2_Test_CS_Task3_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CheckGPTArch_Task3_Prompt2/Best_CS_5000_Task3.pth
python dnn.py CS_5000 3 3 Train_CS_CheckGPTArch_Task3_Prompt3_Test_CS_Task3_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CheckGPTArch_Task3_Prompt3/Best_CS_5000_Task3.pth
python dnn.py CS_5000 3 4 Train_CS_CheckGPTArch_Task3_Prompt4_Test_CS_Task3_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CheckGPTArch_Task3_Prompt4/Best_CS_5000_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 8 - RCH

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_5000 1 1 Train_CS_RCH_Task1_Prompt1 --batchsize 256 --modelid 2
python dnn.py CS_5000 1 2 Train_CS_RCH_Task1_Prompt2 --batchsize 256 --modelid 2
python dnn.py CS_5000 1 3 Train_CS_RCH_Task1_Prompt3 --batchsize 256 --modelid 2
python dnn.py CS_5000 1 4 Train_CS_RCH_Task1_Prompt4 --batchsize 256 --modelid 2

python dnn.py CS_5000 2 1 Train_CS_RCH_Task2_Prompt1 --batchsize 256 --modelid 2
python dnn.py CS_5000 2 2 Train_CS_RCH_Task2_Prompt2 --batchsize 256 --modelid 2
python dnn.py CS_5000 2 3 Train_CS_RCH_Task2_Prompt3 --batchsize 256 --modelid 2
python dnn.py CS_5000 2 4 Train_CS_RCH_Task2_Prompt4 --batchsize 256 --modelid 2

python dnn.py CS_5000 3 1 Train_CS_RCH_Task3_Prompt1 --batchsize 256 --modelid 2
python dnn.py CS_5000 3 2 Train_CS_RCH_Task3_Prompt2 --batchsize 256 --modelid 2
python dnn.py CS_5000 3 3 Train_CS_RCH_Task3_Prompt3 --batchsize 256 --modelid 2
python dnn.py CS_5000 3 4 Train_CS_RCH_Task3_Prompt4 --batchsize 256 --modelid 2

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_5000 1 1 Train_CS_RCH_Task1_Prompt1_Test_CS_Task1_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_RCH_Task1_Prompt1/Best_CS_5000_Task1.pth
python dnn.py CS_5000 1 2 Train_CS_RCH_Task1_Prompt2_Test_CS_Task1_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_RCH_Task1_Prompt2/Best_CS_5000_Task1.pth
python dnn.py CS_5000 1 3 Train_CS_RCH_Task1_Prompt3_Test_CS_Task1_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_RCH_Task1_Prompt3/Best_CS_5000_Task1.pth
python dnn.py CS_5000 1 4 Train_CS_RCH_Task1_Prompt4_Test_CS_Task1_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_RCH_Task1_Prompt4/Best_CS_5000_Task1.pth

python dnn.py CS_5000 2 1 Train_CS_RCH_Task2_Prompt1_Test_CS_Task2_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_RCH_Task2_Prompt1/Best_CS_5000_Task2.pth
python dnn.py CS_5000 2 2 Train_CS_RCH_Task2_Prompt2_Test_CS_Task2_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_RCH_Task2_Prompt2/Best_CS_5000_Task2.pth
python dnn.py CS_5000 2 3 Train_CS_RCH_Task2_Prompt3_Test_CS_Task2_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_RCH_Task2_Prompt3/Best_CS_5000_Task2.pth
python dnn.py CS_5000 2 4 Train_CS_RCH_Task2_Prompt4_Test_CS_Task2_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_RCH_Task2_Prompt4/Best_CS_5000_Task2.pth

python dnn.py CS_5000 3 1 Train_CS_RCH_Task3_Prompt1_Test_CS_Task3_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_RCH_Task3_Prompt1/Best_CS_5000_Task3.pth
python dnn.py CS_5000 3 2 Train_CS_RCH_Task3_Prompt2_Test_CS_Task3_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_RCH_Task3_Prompt2/Best_CS_5000_Task3.pth
python dnn.py CS_5000 3 3 Train_CS_RCH_Task3_Prompt3_Test_CS_Task3_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_RCH_Task3_Prompt3/Best_CS_5000_Task3.pth
python dnn.py CS_5000 3 4 Train_CS_RCH_Task3_Prompt4_Test_CS_Task3_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_RCH_Task3_Prompt4/Best_CS_5000_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 8 - MLP

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_5000 1 1 Train_CS_MLP_Task1_Prompt1 --batchsize 256 --modelid 3
python dnn.py CS_5000 1 2 Train_CS_MLP_Task1_Prompt2 --batchsize 256 --modelid 3
python dnn.py CS_5000 1 3 Train_CS_MLP_Task1_Prompt3 --batchsize 256 --modelid 3
python dnn.py CS_5000 1 4 Train_CS_MLP_Task1_Prompt4 --batchsize 256 --modelid 3

python dnn.py CS_5000 2 1 Train_CS_MLP_Task2_Prompt1 --batchsize 256 --modelid 3
python dnn.py CS_5000 2 2 Train_CS_MLP_Task2_Prompt2 --batchsize 256 --modelid 3
python dnn.py CS_5000 2 3 Train_CS_MLP_Task2_Prompt3 --batchsize 256 --modelid 3
python dnn.py CS_5000 2 4 Train_CS_MLP_Task2_Prompt4 --batchsize 256 --modelid 3

python dnn.py CS_5000 3 1 Train_CS_MLP_Task3_Prompt1 --batchsize 256 --modelid 3
python dnn.py CS_5000 3 2 Train_CS_MLP_Task3_Prompt2 --batchsize 256 --modelid 3
python dnn.py CS_5000 3 3 Train_CS_MLP_Task3_Prompt3 --batchsize 256 --modelid 3
python dnn.py CS_5000 3 4 Train_CS_MLP_Task3_Prompt4 --batchsize 256 --modelid 3

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_5000 1 1 Train_CS_MLP_Task1_Prompt1_Test_CS_Task1_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_MLP_Task1_Prompt1/Best_CS_5000_Task1.pth
python dnn.py CS_5000 1 2 Train_CS_MLP_Task1_Prompt2_Test_CS_Task1_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_MLP_Task1_Prompt2/Best_CS_5000_Task1.pth
python dnn.py CS_5000 1 3 Train_CS_MLP_Task1_Prompt3_Test_CS_Task1_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_MLP_Task1_Prompt3/Best_CS_5000_Task1.pth
python dnn.py CS_5000 1 4 Train_CS_MLP_Task1_Prompt4_Test_CS_Task1_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_MLP_Task1_Prompt4/Best_CS_5000_Task1.pth

python dnn.py CS_5000 2 1 Train_CS_MLP_Task2_Prompt1_Test_CS_Task2_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_MLP_Task2_Prompt1/Best_CS_5000_Task2.pth
python dnn.py CS_5000 2 2 Train_CS_MLP_Task2_Prompt2_Test_CS_Task2_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_MLP_Task2_Prompt2/Best_CS_5000_Task2.pth
python dnn.py CS_5000 2 3 Train_CS_MLP_Task2_Prompt3_Test_CS_Task2_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_MLP_Task2_Prompt3/Best_CS_5000_Task2.pth
python dnn.py CS_5000 2 4 Train_CS_MLP_Task2_Prompt4_Test_CS_Task2_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_MLP_Task2_Prompt4/Best_CS_5000_Task2.pth

python dnn.py CS_5000 3 1 Train_CS_MLP_Task3_Prompt1_Test_CS_Task3_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_MLP_Task3_Prompt1/Best_CS_5000_Task3.pth
python dnn.py CS_5000 3 2 Train_CS_MLP_Task3_Prompt2_Test_CS_Task3_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_MLP_Task3_Prompt2/Best_CS_5000_Task3.pth
python dnn.py CS_5000 3 3 Train_CS_MLP_Task3_Prompt3_Test_CS_Task3_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_MLP_Task3_Prompt3/Best_CS_5000_Task3.pth
python dnn.py CS_5000 3 4 Train_CS_MLP_Task3_Prompt4_Test_CS_Task3_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_MLP_Task3_Prompt4/Best_CS_5000_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 8 - CNN

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_5000 1 1 Train_CS_CNN_Task1_Prompt1 --batchsize 256 --modelid 4
python dnn.py CS_5000 1 2 Train_CS_CNN_Task1_Prompt2 --batchsize 256 --modelid 4
python dnn.py CS_5000 1 3 Train_CS_CNN_Task1_Prompt3 --batchsize 256 --modelid 4
python dnn.py CS_5000 1 4 Train_CS_CNN_Task1_Prompt4 --batchsize 256 --modelid 4

python dnn.py CS_5000 2 1 Train_CS_CNN_Task2_Prompt1 --batchsize 256 --modelid 4
python dnn.py CS_5000 2 2 Train_CS_CNN_Task2_Prompt2 --batchsize 256 --modelid 4
python dnn.py CS_5000 2 3 Train_CS_CNN_Task2_Prompt3 --batchsize 256 --modelid 4
python dnn.py CS_5000 2 4 Train_CS_CNN_Task2_Prompt4 --batchsize 256 --modelid 4

python dnn.py CS_5000 3 1 Train_CS_CNN_Task3_Prompt1 --batchsize 256 --modelid 4
python dnn.py CS_5000 3 2 Train_CS_CNN_Task3_Prompt2 --batchsize 256 --modelid 4
python dnn.py CS_5000 3 3 Train_CS_CNN_Task3_Prompt3 --batchsize 256 --modelid 4
python dnn.py CS_5000 3 4 Train_CS_CNN_Task3_Prompt4 --batchsize 256 --modelid 4

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_5000 1 1 Train_CS_CNN_Task1_Prompt1_Test_CS_Task1_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CNN_Task1_Prompt1/Best_CS_5000_Task1.pth
python dnn.py CS_5000 1 2 Train_CS_CNN_Task1_Prompt2_Test_CS_Task1_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CNN_Task1_Prompt2/Best_CS_5000_Task1.pth
python dnn.py CS_5000 1 3 Train_CS_CNN_Task1_Prompt3_Test_CS_Task1_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CNN_Task1_Prompt3/Best_CS_5000_Task1.pth
python dnn.py CS_5000 1 4 Train_CS_CNN_Task1_Prompt4_Test_CS_Task1_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CNN_Task1_Prompt4/Best_CS_5000_Task1.pth

python dnn.py CS_5000 2 1 Train_CS_CNN_Task2_Prompt1_Test_CS_Task2_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CNN_Task2_Prompt1/Best_CS_5000_Task2.pth
python dnn.py CS_5000 2 2 Train_CS_CNN_Task2_Prompt2_Test_CS_Task2_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CNN_Task2_Prompt2/Best_CS_5000_Task2.pth
python dnn.py CS_5000 2 3 Train_CS_CNN_Task2_Prompt3_Test_CS_Task2_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CNN_Task2_Prompt3/Best_CS_5000_Task2.pth
python dnn.py CS_5000 2 4 Train_CS_CNN_Task2_Prompt4_Test_CS_Task2_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CNN_Task2_Prompt4/Best_CS_5000_Task2.pth

python dnn.py CS_5000 3 1 Train_CS_CNN_Task3_Prompt1_Test_CS_Task3_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CNN_Task3_Prompt1/Best_CS_5000_Task3.pth
python dnn.py CS_5000 3 2 Train_CS_CNN_Task3_Prompt2_Test_CS_Task3_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CNN_Task3_Prompt2/Best_CS_5000_Task3.pth
python dnn.py CS_5000 3 3 Train_CS_CNN_Task3_Prompt3_Test_CS_Task3_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CNN_Task3_Prompt3/Best_CS_5000_Task3.pth
python dnn.py CS_5000 3 4 Train_CS_CNN_Task3_Prompt4_Test_CS_Task3_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CNN_Task3_Prompt4/Best_CS_5000_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 8 - BiLSTM w/o attention

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_5000 1 1 Train_CS_BiLSTMwoAttention_Task1_Prompt1 --batchsize 256 --modelid 1
python dnn.py CS_5000 1 2 Train_CS_BiLSTMwoAttention_Task1_Prompt2 --batchsize 256 --modelid 1
python dnn.py CS_5000 1 3 Train_CS_BiLSTMwoAttention_Task1_Prompt3 --batchsize 256 --modelid 1
python dnn.py CS_5000 1 4 Train_CS_BiLSTMwoAttention_Task1_Prompt4 --batchsize 256 --modelid 1

python dnn.py CS_5000 2 1 Train_CS_BiLSTMwoAttention_Task2_Prompt1 --batchsize 256 --modelid 1
python dnn.py CS_5000 2 2 Train_CS_BiLSTMwoAttention_Task2_Prompt2 --batchsize 256 --modelid 1
python dnn.py CS_5000 2 3 Train_CS_BiLSTMwoAttention_Task2_Prompt3 --batchsize 256 --modelid 1
python dnn.py CS_5000 2 4 Train_CS_BiLSTMwoAttention_Task2_Prompt4 --batchsize 256 --modelid 1

python dnn.py CS_5000 3 1 Train_CS_BiLSTMwoAttention_Task3_Prompt1 --batchsize 256 --modelid 1
python dnn.py CS_5000 3 2 Train_CS_BiLSTMwoAttention_Task3_Prompt2 --batchsize 256 --modelid 1
python dnn.py CS_5000 3 3 Train_CS_BiLSTMwoAttention_Task3_Prompt3 --batchsize 256 --modelid 1
python dnn.py CS_5000 3 4 Train_CS_BiLSTMwoAttention_Task3_Prompt4 --batchsize 256 --modelid 1

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_5000 1 1 Train_CS_BiLSTMwoAttention_Task1_Prompt1_Test_CS_Task1_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_BiLSTMwoAttention_Task1_Prompt1/Best_CS_5000_Task1.pth
python dnn.py CS_5000 1 2 Train_CS_BiLSTMwoAttention_Task1_Prompt2_Test_CS_Task1_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_BiLSTMwoAttention_Task1_Prompt2/Best_CS_5000_Task1.pth
python dnn.py CS_5000 1 3 Train_CS_BiLSTMwoAttention_Task1_Prompt3_Test_CS_Task1_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_BiLSTMwoAttention_Task1_Prompt3/Best_CS_5000_Task1.pth
python dnn.py CS_5000 1 4 Train_CS_BiLSTMwoAttention_Task1_Prompt4_Test_CS_Task1_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_BiLSTMwoAttention_Task1_Prompt4/Best_CS_5000_Task1.pth

python dnn.py CS_5000 2 1 Train_CS_BiLSTMwoAttention_Task2_Prompt1_Test_CS_Task2_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_BiLSTMwoAttention_Task2_Prompt1/Best_CS_5000_Task2.pth
python dnn.py CS_5000 2 2 Train_CS_BiLSTMwoAttention_Task2_Prompt2_Test_CS_Task2_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_BiLSTMwoAttention_Task2_Prompt2/Best_CS_5000_Task2.pth
python dnn.py CS_5000 2 3 Train_CS_BiLSTMwoAttention_Task2_Prompt3_Test_CS_Task2_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_BiLSTMwoAttention_Task2_Prompt3/Best_CS_5000_Task2.pth
python dnn.py CS_5000 2 4 Train_CS_BiLSTMwoAttention_Task2_Prompt4_Test_CS_Task2_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_BiLSTMwoAttention_Task2_Prompt4/Best_CS_5000_Task2.pth

python dnn.py CS_5000 3 1 Train_CS_BiLSTMwoAttention_Task3_Prompt1_Test_CS_Task3_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_BiLSTMwoAttention_Task3_Prompt1/Best_CS_5000_Task3.pth
python dnn.py CS_5000 3 2 Train_CS_BiLSTMwoAttention_Task3_Prompt2_Test_CS_Task3_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_BiLSTMwoAttention_Task3_Prompt2/Best_CS_5000_Task3.pth
python dnn.py CS_5000 3 3 Train_CS_BiLSTMwoAttention_Task3_Prompt3_Test_CS_Task3_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_BiLSTMwoAttention_Task3_Prompt3/Best_CS_5000_Task3.pth
python dnn.py CS_5000 3 4 Train_CS_BiLSTMwoAttention_Task3_Prompt4_Test_CS_Task3_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_BiLSTMwoAttention_Task3_Prompt4/Best_CS_5000_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 9 - Cross Prompt

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_5000 1 0 Train_CS_CheckGPTArch_Task1_Prompt1234_Test_CS_Task1_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/CS_Task1_Prompt_1234/Best_CS_Task1.pth
python dnn.py CS_5000 2 0 Train_CS_CheckGPTArch_Task2_Prompt1234_Test_CS_Task2_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/CS_Task2_Prompt1234/Best_CS_Task2.pth
python dnn.py CS_5000 3 0 Train_CS_CheckGPTArch_Task3_Prompt1234_Test_CS_Task3_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/CS_Task3_Prompt1234/Best_CS_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 9 - Cross Prompt,Domain

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_5000 1 0 Train_CSPHXHSS_CheckGPTArch_Task1_Prompt1234_Test_CS_Task1_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/ALL_Task1_Prompt1234/Best_ALL_Task1.pth
python dnn.py CS_5000 2 0 Train_CSPHXHSS_CheckGPTArch_Task2_Prompt1234_Test_CS_Task2_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/ALL_Task2_Prompt1234/Best_ALL_Task2.pth
python dnn.py CS_5000 3 0 Train_CSPHXHSS_CheckGPTArch_Task3_Prompt1234_Test_CS_Task3_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/ALL_Task3_Prompt1234/Best_ALL_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 9 - Cross Prompt,Domain,Task

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_5000 1 0 Train_CSPHXHSS_CheckGPTArch_Task123_Prompt1234_Test_CS_Task1_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/ALL_Task123_Prompt1234/Best_ALL_Task0.pth
python dnn.py CS_5000 2 0 Train_CSPHXHSS_CheckGPTArch_Task123_Prompt1234_Test_CS_Task2_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/ALL_Task123_Prompt1234/Best_ALL_Task0.pth
python dnn.py CS_5000 3 0 Train_CSPHXHSS_CheckGPTArch_Task123_Prompt1234_Test_CS_Task3_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/ALL_Task123_Prompt1234/Best_ALL_Task0.pth

### Table 10 - Prompt 234

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_5000 1 234 Train_CS_CheckGPTArch_Task1_Prompt234 --batchsize 256
python dnn.py CS_5000 2 234 Train_CS_CheckGPTArch_Task2_Prompt234 --batchsize 256
python dnn.py CS_5000 3 234 Train_CS_CheckGPTArch_Task3_Prompt234 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_5000 1 1 Train_CS_CheckGPTArch_Task1_Prompt234_Test_CS_Task1_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CheckGPTArch_Task1_Prompt234/Best_CS_5000_Task1.pth
python dnn.py CS_5000 2 1 Train_CS_CheckGPTArch_Task2_Prompt234_Test_CS_Task2_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CheckGPTArch_Task2_Prompt234/Best_CS_5000_Task2.pth
python dnn.py CS_5000 3 1 Train_CS_CheckGPTArch_Task3_Prompt234_Test_CS_Task3_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CheckGPTArch_Task3_Prompt234/Best_CS_5000_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 10 - Prompt 134

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_5000 1 134 Train_CS_CheckGPTArch_Task1_Prompt134 --batchsize 256
python dnn.py CS_5000 2 134 Train_CS_CheckGPTArch_Task2_Prompt134 --batchsize 256
python dnn.py CS_5000 3 134 Train_CS_CheckGPTArch_Task3_Prompt134 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_5000 1 2 Train_CS_CheckGPTArch_Task1_Prompt134_Test_CS_Task1_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CheckGPTArch_Task1_Prompt134/Best_CS_5000_Task1.pth
python dnn.py CS_5000 2 2 Train_CS_CheckGPTArch_Task2_Prompt134_Test_CS_Task2_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CheckGPTArch_Task2_Prompt134/Best_CS_5000_Task2.pth
python dnn.py CS_5000 3 2 Train_CS_CheckGPTArch_Task3_Prompt134_Test_CS_Task3_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CheckGPTArch_Task3_Prompt134/Best_CS_5000_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 10 - Prompt 124

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_5000 1 124 Train_CS_CheckGPTArch_Task1_Prompt124 --batchsize 256
python dnn.py CS_5000 2 124 Train_CS_CheckGPTArch_Task2_Prompt124 --batchsize 256
python dnn.py CS_5000 3 124 Train_CS_CheckGPTArch_Task3_Prompt124 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_5000 1 3 Train_CS_CheckGPTArch_Task1_Prompt124_Test_CS_Task1_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CheckGPTArch_Task1_Prompt124/Best_CS_5000_Task1.pth
python dnn.py CS_5000 2 3 Train_CS_CheckGPTArch_Task2_Prompt124_Test_CS_Task2_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CheckGPTArch_Task2_Prompt124/Best_CS_5000_Task2.pth
python dnn.py CS_5000 3 3 Train_CS_CheckGPTArch_Task3_Prompt124_Test_CS_Task3_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CheckGPTArch_Task3_Prompt124/Best_CS_5000_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 10 - Prompt 123

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_5000 1 123 Train_CS_CheckGPTArch_Task1_Prompt123 --batchsize 256
python dnn.py CS_5000 2 123 Train_CS_CheckGPTArch_Task2_Prompt123 --batchsize 256
python dnn.py CS_5000 3 123 Train_CS_CheckGPTArch_Task3_Prompt123 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py CS_5000 1 4 Train_CS_CheckGPTArch_Task1_Prompt123_Test_CS_Task1_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CheckGPTArch_Task1_Prompt123/Best_CS_5000_Task1.pth
python dnn.py CS_5000 2 4 Train_CS_CheckGPTArch_Task2_Prompt123_Test_CS_Task2_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CheckGPTArch_Task2_Prompt123/Best_CS_5000_Task2.pth
python dnn.py CS_5000 3 4 Train_CS_CheckGPTArch_Task3_Prompt123_Test_CS_Task3_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_CS_CheckGPTArch_Task3_Prompt123/Best_CS_5000_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Feature Clean-up

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/
shopt -s extglob
rm -rf embeddings/!(README.md)

## PHX - 5000

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python features.py PHX 1 1 --gpt 0 --number 5000
python features.py PHX 2 1 --gpt 0 --number 5000

python features.py PHX 1 1 --gpt 1 --number 5000
python features.py PHX 1 2 --gpt 1 --number 5000
python features.py PHX 1 3 --gpt 1 --number 5000
python features.py PHX 1 4 --gpt 1 --number 5000

python features.py PHX 2 1 --gpt 1 --number 5000
python features.py PHX 2 2 --gpt 1 --number 5000
python features.py PHX 2 3 --gpt 1 --number 5000
python features.py PHX 2 4 --gpt 1 --number 5000

python features.py PHX 3 1 --gpt 1 --number 5000
python features.py PHX 3 2 --gpt 1 --number 5000
python features.py PHX 3 3 --gpt 1 --number 5000
python features.py PHX 3 4 --gpt 1 --number 5000

In [ ]:
elapsed_time(start)
start = now()

### Table 7 & 8 - CheckGPT

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_5000 1 1 Train_PHX_CheckGPTArch_Task1_Prompt1 --batchsize 256
python dnn.py PHX_5000 1 2 Train_PHX_CheckGPTArch_Task1_Prompt2 --batchsize 256
python dnn.py PHX_5000 1 3 Train_PHX_CheckGPTArch_Task1_Prompt3 --batchsize 256
python dnn.py PHX_5000 1 4 Train_PHX_CheckGPTArch_Task1_Prompt4 --batchsize 256

python dnn.py PHX_5000 2 1 Train_PHX_CheckGPTArch_Task2_Prompt1 --batchsize 256
python dnn.py PHX_5000 2 2 Train_PHX_CheckGPTArch_Task2_Prompt2 --batchsize 256
python dnn.py PHX_5000 2 3 Train_PHX_CheckGPTArch_Task2_Prompt3 --batchsize 256
python dnn.py PHX_5000 2 4 Train_PHX_CheckGPTArch_Task2_Prompt4 --batchsize 256

python dnn.py PHX_5000 3 1 Train_PHX_CheckGPTArch_Task3_Prompt1 --batchsize 256
python dnn.py PHX_5000 3 2 Train_PHX_CheckGPTArch_Task3_Prompt2 --batchsize 256
python dnn.py PHX_5000 3 3 Train_PHX_CheckGPTArch_Task3_Prompt3 --batchsize 256
python dnn.py PHX_5000 3 4 Train_PHX_CheckGPTArch_Task3_Prompt4 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_5000 1 1 Train_PHX_CheckGPTArch_Task1_Prompt1_Test_PHX_Task1_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CheckGPTArch_Task1_Prompt1/Best_PHX_5000_Task1.pth
python dnn.py PHX_5000 1 2 Train_PHX_CheckGPTArch_Task1_Prompt2_Test_PHX_Task1_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CheckGPTArch_Task1_Prompt2/Best_PHX_5000_Task1.pth
python dnn.py PHX_5000 1 3 Train_PHX_CheckGPTArch_Task1_Prompt3_Test_PHX_Task1_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CheckGPTArch_Task1_Prompt3/Best_PHX_5000_Task1.pth
python dnn.py PHX_5000 1 4 Train_PHX_CheckGPTArch_Task1_Prompt4_Test_PHX_Task1_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CheckGPTArch_Task1_Prompt4/Best_PHX_5000_Task1.pth

python dnn.py PHX_5000 2 1 Train_PHX_CheckGPTArch_Task2_Prompt1_Test_PHX_Task2_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CheckGPTArch_Task2_Prompt1/Best_PHX_5000_Task2.pth
python dnn.py PHX_5000 2 2 Train_PHX_CheckGPTArch_Task2_Prompt2_Test_PHX_Task2_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CheckGPTArch_Task2_Prompt2/Best_PHX_5000_Task2.pth
python dnn.py PHX_5000 2 3 Train_PHX_CheckGPTArch_Task2_Prompt3_Test_PHX_Task2_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CheckGPTArch_Task2_Prompt3/Best_PHX_5000_Task2.pth
python dnn.py PHX_5000 2 4 Train_PHX_CheckGPTArch_Task2_Prompt4_Test_PHX_Task2_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CheckGPTArch_Task2_Prompt4/Best_PHX_5000_Task2.pth

python dnn.py PHX_5000 3 1 Train_PHX_CheckGPTArch_Task3_Prompt1_Test_PHX_Task3_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CheckGPTArch_Task3_Prompt1/Best_PHX_5000_Task3.pth
python dnn.py PHX_5000 3 2 Train_PHX_CheckGPTArch_Task3_Prompt2_Test_PHX_Task3_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CheckGPTArch_Task3_Prompt2/Best_PHX_5000_Task3.pth
python dnn.py PHX_5000 3 3 Train_PHX_CheckGPTArch_Task3_Prompt3_Test_PHX_Task3_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CheckGPTArch_Task3_Prompt3/Best_PHX_5000_Task3.pth
python dnn.py PHX_5000 3 4 Train_PHX_CheckGPTArch_Task3_Prompt4_Test_PHX_Task3_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CheckGPTArch_Task3_Prompt4/Best_PHX_5000_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 8 - RCH

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_5000 1 1 Train_PHX_RCH_Task1_Prompt1 --batchsize 256 --modelid 2
python dnn.py PHX_5000 1 2 Train_PHX_RCH_Task1_Prompt2 --batchsize 256 --modelid 2
python dnn.py PHX_5000 1 3 Train_PHX_RCH_Task1_Prompt3 --batchsize 256 --modelid 2
python dnn.py PHX_5000 1 4 Train_PHX_RCH_Task1_Prompt4 --batchsize 256 --modelid 2

python dnn.py PHX_5000 2 1 Train_PHX_RCH_Task2_Prompt1 --batchsize 256 --modelid 2
python dnn.py PHX_5000 2 2 Train_PHX_RCH_Task2_Prompt2 --batchsize 256 --modelid 2
python dnn.py PHX_5000 2 3 Train_PHX_RCH_Task2_Prompt3 --batchsize 256 --modelid 2
python dnn.py PHX_5000 2 4 Train_PHX_RCH_Task2_Prompt4 --batchsize 256 --modelid 2

python dnn.py PHX_5000 3 1 Train_PHX_RCH_Task3_Prompt1 --batchsize 256 --modelid 2
python dnn.py PHX_5000 3 2 Train_PHX_RCH_Task3_Prompt2 --batchsize 256 --modelid 2
python dnn.py PHX_5000 3 3 Train_PHX_RCH_Task3_Prompt3 --batchsize 256 --modelid 2
python dnn.py PHX_5000 3 4 Train_PHX_RCH_Task3_Prompt4 --batchsize 256 --modelid 2

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_5000 1 1 Train_PHX_RCH_Task1_Prompt1_Test_PHX_Task1_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_RCH_Task1_Prompt1/Best_PHX_5000_Task1.pth
python dnn.py PHX_5000 1 2 Train_PHX_RCH_Task1_Prompt2_Test_PHX_Task1_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_RCH_Task1_Prompt2/Best_PHX_5000_Task1.pth
python dnn.py PHX_5000 1 3 Train_PHX_RCH_Task1_Prompt3_Test_PHX_Task1_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_RCH_Task1_Prompt3/Best_PHX_5000_Task1.pth
python dnn.py PHX_5000 1 4 Train_PHX_RCH_Task1_Prompt4_Test_PHX_Task1_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_RCH_Task1_Prompt4/Best_PHX_5000_Task1.pth

python dnn.py PHX_5000 2 1 Train_PHX_RCH_Task2_Prompt1_Test_PHX_Task2_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_RCH_Task2_Prompt1/Best_PHX_5000_Task2.pth
python dnn.py PHX_5000 2 2 Train_PHX_RCH_Task2_Prompt2_Test_PHX_Task2_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_RCH_Task2_Prompt2/Best_PHX_5000_Task2.pth
python dnn.py PHX_5000 2 3 Train_PHX_RCH_Task2_Prompt3_Test_PHX_Task2_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_RCH_Task2_Prompt3/Best_PHX_5000_Task2.pth
python dnn.py PHX_5000 2 4 Train_PHX_RCH_Task2_Prompt4_Test_PHX_Task2_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_RCH_Task2_Prompt4/Best_PHX_5000_Task2.pth

python dnn.py PHX_5000 3 1 Train_PHX_RCH_Task3_Prompt1_Test_PHX_Task3_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_RCH_Task3_Prompt1/Best_PHX_5000_Task3.pth
python dnn.py PHX_5000 3 2 Train_PHX_RCH_Task3_Prompt2_Test_PHX_Task3_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_RCH_Task3_Prompt2/Best_PHX_5000_Task3.pth
python dnn.py PHX_5000 3 3 Train_PHX_RCH_Task3_Prompt3_Test_PHX_Task3_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_RCH_Task3_Prompt3/Best_PHX_5000_Task3.pth
python dnn.py PHX_5000 3 4 Train_PHX_RCH_Task3_Prompt4_Test_PHX_Task3_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_RCH_Task3_Prompt4/Best_PHX_5000_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 8 - MLP

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_5000 1 1 Train_PHX_MLP_Task1_Prompt1 --batchsize 256 --modelid 3
python dnn.py PHX_5000 1 2 Train_PHX_MLP_Task1_Prompt2 --batchsize 256 --modelid 3
python dnn.py PHX_5000 1 3 Train_PHX_MLP_Task1_Prompt3 --batchsize 256 --modelid 3
python dnn.py PHX_5000 1 4 Train_PHX_MLP_Task1_Prompt4 --batchsize 256 --modelid 3

python dnn.py PHX_5000 2 1 Train_PHX_MLP_Task2_Prompt1 --batchsize 256 --modelid 3
python dnn.py PHX_5000 2 2 Train_PHX_MLP_Task2_Prompt2 --batchsize 256 --modelid 3
python dnn.py PHX_5000 2 3 Train_PHX_MLP_Task2_Prompt3 --batchsize 256 --modelid 3
python dnn.py PHX_5000 2 4 Train_PHX_MLP_Task2_Prompt4 --batchsize 256 --modelid 3

python dnn.py PHX_5000 3 1 Train_PHX_MLP_Task3_Prompt1 --batchsize 256 --modelid 3
python dnn.py PHX_5000 3 2 Train_PHX_MLP_Task3_Prompt2 --batchsize 256 --modelid 3
python dnn.py PHX_5000 3 3 Train_PHX_MLP_Task3_Prompt3 --batchsize 256 --modelid 3
python dnn.py PHX_5000 3 4 Train_PHX_MLP_Task3_Prompt4 --batchsize 256 --modelid 3

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_5000 1 1 Train_PHX_MLP_Task1_Prompt1_Test_PHX_Task1_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_MLP_Task1_Prompt1/Best_PHX_5000_Task1.pth
python dnn.py PHX_5000 1 2 Train_PHX_MLP_Task1_Prompt2_Test_PHX_Task1_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_MLP_Task1_Prompt2/Best_PHX_5000_Task1.pth
python dnn.py PHX_5000 1 3 Train_PHX_MLP_Task1_Prompt3_Test_PHX_Task1_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_MLP_Task1_Prompt3/Best_PHX_5000_Task1.pth
python dnn.py PHX_5000 1 4 Train_PHX_MLP_Task1_Prompt4_Test_PHX_Task1_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_MLP_Task1_Prompt4/Best_PHX_5000_Task1.pth

python dnn.py PHX_5000 2 1 Train_PHX_MLP_Task2_Prompt1_Test_PHX_Task2_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_MLP_Task2_Prompt1/Best_PHX_5000_Task2.pth
python dnn.py PHX_5000 2 2 Train_PHX_MLP_Task2_Prompt2_Test_PHX_Task2_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_MLP_Task2_Prompt2/Best_PHX_5000_Task2.pth
python dnn.py PHX_5000 2 3 Train_PHX_MLP_Task2_Prompt3_Test_PHX_Task2_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_MLP_Task2_Prompt3/Best_PHX_5000_Task2.pth
python dnn.py PHX_5000 2 4 Train_PHX_MLP_Task2_Prompt4_Test_PHX_Task2_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_MLP_Task2_Prompt4/Best_PHX_5000_Task2.pth

python dnn.py PHX_5000 3 1 Train_PHX_MLP_Task3_Prompt1_Test_PHX_Task3_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_MLP_Task3_Prompt1/Best_PHX_5000_Task3.pth
python dnn.py PHX_5000 3 2 Train_PHX_MLP_Task3_Prompt2_Test_PHX_Task3_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_MLP_Task3_Prompt2/Best_PHX_5000_Task3.pth
python dnn.py PHX_5000 3 3 Train_PHX_MLP_Task3_Prompt3_Test_PHX_Task3_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_MLP_Task3_Prompt3/Best_PHX_5000_Task3.pth
python dnn.py PHX_5000 3 4 Train_PHX_MLP_Task3_Prompt4_Test_PHX_Task3_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_MLP_Task3_Prompt4/Best_PHX_5000_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 8 - CNN

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_5000 1 1 Train_PHX_CNN_Task1_Prompt1 --batchsize 256 --modelid 4
python dnn.py PHX_5000 1 2 Train_PHX_CNN_Task1_Prompt2 --batchsize 256 --modelid 4
python dnn.py PHX_5000 1 3 Train_PHX_CNN_Task1_Prompt3 --batchsize 256 --modelid 4
python dnn.py PHX_5000 1 4 Train_PHX_CNN_Task1_Prompt4 --batchsize 256 --modelid 4

python dnn.py PHX_5000 2 1 Train_PHX_CNN_Task2_Prompt1 --batchsize 256 --modelid 4
python dnn.py PHX_5000 2 2 Train_PHX_CNN_Task2_Prompt2 --batchsize 256 --modelid 4
python dnn.py PHX_5000 2 3 Train_PHX_CNN_Task2_Prompt3 --batchsize 256 --modelid 4
python dnn.py PHX_5000 2 4 Train_PHX_CNN_Task2_Prompt4 --batchsize 256 --modelid 4

python dnn.py PHX_5000 3 1 Train_PHX_CNN_Task3_Prompt1 --batchsize 256 --modelid 4
python dnn.py PHX_5000 3 2 Train_PHX_CNN_Task3_Prompt2 --batchsize 256 --modelid 4
python dnn.py PHX_5000 3 3 Train_PHX_CNN_Task3_Prompt3 --batchsize 256 --modelid 4
python dnn.py PHX_5000 3 4 Train_PHX_CNN_Task3_Prompt4 --batchsize 256 --modelid 4

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_5000 1 1 Train_PHX_CNN_Task1_Prompt1_Test_PHX_Task1_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CNN_Task1_Prompt1/Best_PHX_5000_Task1.pth
python dnn.py PHX_5000 1 2 Train_PHX_CNN_Task1_Prompt2_Test_PHX_Task1_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CNN_Task1_Prompt2/Best_PHX_5000_Task1.pth
python dnn.py PHX_5000 1 3 Train_PHX_CNN_Task1_Prompt3_Test_PHX_Task1_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CNN_Task1_Prompt3/Best_PHX_5000_Task1.pth
python dnn.py PHX_5000 1 4 Train_PHX_CNN_Task1_Prompt4_Test_PHX_Task1_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CNN_Task1_Prompt4/Best_PHX_5000_Task1.pth

python dnn.py PHX_5000 2 1 Train_PHX_CNN_Task2_Prompt1_Test_PHX_Task2_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CNN_Task2_Prompt1/Best_PHX_5000_Task2.pth
python dnn.py PHX_5000 2 2 Train_PHX_CNN_Task2_Prompt2_Test_PHX_Task2_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CNN_Task2_Prompt2/Best_PHX_5000_Task2.pth
python dnn.py PHX_5000 2 3 Train_PHX_CNN_Task2_Prompt3_Test_PHX_Task2_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CNN_Task2_Prompt3/Best_PHX_5000_Task2.pth
python dnn.py PHX_5000 2 4 Train_PHX_CNN_Task2_Prompt4_Test_PHX_Task2_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CNN_Task2_Prompt4/Best_PHX_5000_Task2.pth

python dnn.py PHX_5000 3 1 Train_PHX_CNN_Task3_Prompt1_Test_PHX_Task3_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CNN_Task3_Prompt1/Best_PHX_5000_Task3.pth
python dnn.py PHX_5000 3 2 Train_PHX_CNN_Task3_Prompt2_Test_PHX_Task3_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CNN_Task3_Prompt2/Best_PHX_5000_Task3.pth
python dnn.py PHX_5000 3 3 Train_PHX_CNN_Task3_Prompt3_Test_PHX_Task3_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CNN_Task3_Prompt3/Best_PHX_5000_Task3.pth
python dnn.py PHX_5000 3 4 Train_PHX_CNN_Task3_Prompt4_Test_PHX_Task3_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CNN_Task3_Prompt4/Best_PHX_5000_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 8 - BiLSTM w/o attention

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_5000 1 1 Train_PHX_BiLSTMwoAttention_Task1_Prompt1 --batchsize 256 --modelid 1
python dnn.py PHX_5000 1 2 Train_PHX_BiLSTMwoAttention_Task1_Prompt2 --batchsize 256 --modelid 1
python dnn.py PHX_5000 1 3 Train_PHX_BiLSTMwoAttention_Task1_Prompt3 --batchsize 256 --modelid 1
python dnn.py PHX_5000 1 4 Train_PHX_BiLSTMwoAttention_Task1_Prompt4 --batchsize 256 --modelid 1

python dnn.py PHX_5000 2 1 Train_PHX_BiLSTMwoAttention_Task2_Prompt1 --batchsize 256 --modelid 1
python dnn.py PHX_5000 2 2 Train_PHX_BiLSTMwoAttention_Task2_Prompt2 --batchsize 256 --modelid 1
python dnn.py PHX_5000 2 3 Train_PHX_BiLSTMwoAttention_Task2_Prompt3 --batchsize 256 --modelid 1
python dnn.py PHX_5000 2 4 Train_PHX_BiLSTMwoAttention_Task2_Prompt4 --batchsize 256 --modelid 1

python dnn.py PHX_5000 3 1 Train_PHX_BiLSTMwoAttention_Task3_Prompt1 --batchsize 256 --modelid 1
python dnn.py PHX_5000 3 2 Train_PHX_BiLSTMwoAttention_Task3_Prompt2 --batchsize 256 --modelid 1
python dnn.py PHX_5000 3 3 Train_PHX_BiLSTMwoAttention_Task3_Prompt3 --batchsize 256 --modelid 1
python dnn.py PHX_5000 3 4 Train_PHX_BiLSTMwoAttention_Task3_Prompt4 --batchsize 256 --modelid 1

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_5000 1 1 Train_PHX_BiLSTMwoAttention_Task1_Prompt1_Test_PHX_Task1_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_BiLSTMwoAttention_Task1_Prompt1/Best_PHX_5000_Task1.pth
python dnn.py PHX_5000 1 2 Train_PHX_BiLSTMwoAttention_Task1_Prompt2_Test_PHX_Task1_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_BiLSTMwoAttention_Task1_Prompt2/Best_PHX_5000_Task1.pth
python dnn.py PHX_5000 1 3 Train_PHX_BiLSTMwoAttention_Task1_Prompt3_Test_PHX_Task1_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_BiLSTMwoAttention_Task1_Prompt3/Best_PHX_5000_Task1.pth
python dnn.py PHX_5000 1 4 Train_PHX_BiLSTMwoAttention_Task1_Prompt4_Test_PHX_Task1_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_BiLSTMwoAttention_Task1_Prompt4/Best_PHX_5000_Task1.pth

python dnn.py PHX_5000 2 1 Train_PHX_BiLSTMwoAttention_Task2_Prompt1_Test_PHX_Task2_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_BiLSTMwoAttention_Task2_Prompt1/Best_PHX_5000_Task2.pth
python dnn.py PHX_5000 2 2 Train_PHX_BiLSTMwoAttention_Task2_Prompt2_Test_PHX_Task2_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_BiLSTMwoAttention_Task2_Prompt2/Best_PHX_5000_Task2.pth
python dnn.py PHX_5000 2 3 Train_PHX_BiLSTMwoAttention_Task2_Prompt3_Test_PHX_Task2_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_BiLSTMwoAttention_Task2_Prompt3/Best_PHX_5000_Task2.pth
python dnn.py PHX_5000 2 4 Train_PHX_BiLSTMwoAttention_Task2_Prompt4_Test_PHX_Task2_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_BiLSTMwoAttention_Task2_Prompt4/Best_PHX_5000_Task2.pth

python dnn.py PHX_5000 3 1 Train_PHX_BiLSTMwoAttention_Task3_Prompt1_Test_PHX_Task3_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_BiLSTMwoAttention_Task3_Prompt1/Best_PHX_5000_Task3.pth
python dnn.py PHX_5000 3 2 Train_PHX_BiLSTMwoAttention_Task3_Prompt2_Test_PHX_Task3_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_BiLSTMwoAttention_Task3_Prompt2/Best_PHX_5000_Task3.pth
python dnn.py PHX_5000 3 3 Train_PHX_BiLSTMwoAttention_Task3_Prompt3_Test_PHX_Task3_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_BiLSTMwoAttention_Task3_Prompt3/Best_PHX_5000_Task3.pth
python dnn.py PHX_5000 3 4 Train_PHX_BiLSTMwoAttention_Task3_Prompt4_Test_PHX_Task3_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_BiLSTMwoAttention_Task3_Prompt4/Best_PHX_5000_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 9 - Cross Prompt

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_5000 1 0 Train_PHX_CheckGPTArch_Task1_Prompt1234_Test_PHX_Task1_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/PHX_Task1_Prompt1234/Best_PHY_Task1.pth
python dnn.py PHX_5000 2 0 Train_PHX_CheckGPTArch_Task2_Prompt1234_Test_PHX_Task2_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/PHX_Task2_Prompt1234/Best_PHY_Task2.pth
python dnn.py PHX_5000 3 0 Train_PHX_CheckGPTArch_Task3_Prompt1234_Test_PHX_Task3_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/PHX_Task3_Prompt1234/Best_PHY_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 9 - Cross Prompt,Domain

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_5000 1 0 Train_CSPHXHSS_CheckGPTArch_Task1_Prompt1234_Test_PHX_Task1_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/ALL_Task1_Prompt1234/Best_ALL_Task1.pth
python dnn.py PHX_5000 2 0 Train_CSPHXHSS_CheckGPTArch_Task2_Prompt1234_Test_PHX_Task2_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/ALL_Task2_Prompt1234/Best_ALL_Task2.pth
python dnn.py PHX_5000 3 0 Train_CSPHXHSS_CheckGPTArch_Task3_Prompt1234_Test_PHX_Task3_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/ALL_Task3_Prompt1234/Best_ALL_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 9 - Cross Prompt,Domain,Task

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_5000 1 0 Train_CSPHXHSS_CheckGPTArch_Task123_Prompt1234_Test_PHX_Task1_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/ALL_Task123_Prompt1234/Best_ALL_Task0.pth
python dnn.py PHX_5000 2 0 Train_CSPHXHSS_CheckGPTArch_Task123_Prompt1234_Test_PHX_Task2_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/ALL_Task123_Prompt1234/Best_ALL_Task0.pth
python dnn.py PHX_5000 3 0 Train_CSPHXHSS_CheckGPTArch_Task123_Prompt1234_Test_PHX_Task3_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/ALL_Task123_Prompt1234/Best_ALL_Task0.pth

### Table 10 - Prompt 234

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_5000 1 234 Train_PHX_CheckGPTArch_Task1_Prompt234 --batchsize 256
python dnn.py PHX_5000 2 234 Train_PHX_CheckGPTArch_Task2_Prompt234 --batchsize 256
python dnn.py PHX_5000 3 234 Train_PHX_CheckGPTArch_Task3_Prompt234 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_5000 1 1 Train_PHX_CheckGPTArch_Task1_Prompt234_Test_PHX_Task1_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CheckGPTArch_Task1_Prompt234/Best_PHX_5000_Task1.pth
python dnn.py PHX_5000 2 1 Train_PHX_CheckGPTArch_Task2_Prompt234_Test_PHX_Task2_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CheckGPTArch_Task2_Prompt234/Best_PHX_5000_Task2.pth
python dnn.py PHX_5000 3 1 Train_PHX_CheckGPTArch_Task3_Prompt234_Test_PHX_Task3_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CheckGPTArch_Task3_Prompt234/Best_PHX_5000_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 10 - Prompt 134

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_5000 1 134 Train_PHX_CheckGPTArch_Task1_Prompt134 --batchsize 256
python dnn.py PHX_5000 2 134 Train_PHX_CheckGPTArch_Task2_Prompt134 --batchsize 256
python dnn.py PHX_5000 3 134 Train_PHX_CheckGPTArch_Task3_Prompt134 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_5000 1 2 Train_PHX_CheckGPTArch_Task1_Prompt134_Test_PHX_Task1_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CheckGPTArch_Task1_Prompt134/Best_PHX_5000_Task1.pth
python dnn.py PHX_5000 2 2 Train_PHX_CheckGPTArch_Task2_Prompt134_Test_PHX_Task2_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CheckGPTArch_Task2_Prompt134/Best_PHX_5000_Task2.pth
python dnn.py PHX_5000 3 2 Train_PHX_CheckGPTArch_Task3_Prompt134_Test_PHX_Task3_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CheckGPTArch_Task3_Prompt134/Best_PHX_5000_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 10 - Prompt 124

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_5000 1 124 Train_PHX_CheckGPTArch_Task1_Prompt124 --batchsize 256
python dnn.py PHX_5000 2 124 Train_PHX_CheckGPTArch_Task2_Prompt124 --batchsize 256
python dnn.py PHX_5000 3 124 Train_PHX_CheckGPTArch_Task3_Prompt124 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_5000 1 3 Train_PHX_CheckGPTArch_Task1_Prompt124_Test_PHX_Task1_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CheckGPTArch_Task1_Prompt124/Best_PHX_5000_Task1.pth
python dnn.py PHX_5000 2 3 Train_PHX_CheckGPTArch_Task2_Prompt124_Test_PHX_Task2_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CheckGPTArch_Task2_Prompt124/Best_PHX_5000_Task2.pth
python dnn.py PHX_5000 3 3 Train_PHX_CheckGPTArch_Task3_Prompt124_Test_PHX_Task3_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CheckGPTArch_Task3_Prompt124/Best_PHX_5000_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 10 - Prompt 123

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_5000 1 123 Train_PHX_CheckGPTArch_Task1_Prompt123 --batchsize 256
python dnn.py PHX_5000 2 123 Train_PHX_CheckGPTArch_Task2_Prompt123 --batchsize 256
python dnn.py PHX_5000 3 123 Train_PHX_CheckGPTArch_Task3_Prompt123 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py PHX_5000 1 4 Train_PHX_CheckGPTArch_Task1_Prompt123_Test_PHX_Task1_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CheckGPTArch_Task1_Prompt123/Best_PHX_5000_Task1.pth
python dnn.py PHX_5000 2 4 Train_PHX_CheckGPTArch_Task2_Prompt123_Test_PHX_Task2_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CheckGPTArch_Task2_Prompt123/Best_PHX_5000_Task2.pth
python dnn.py PHX_5000 3 4 Train_PHX_CheckGPTArch_Task3_Prompt123_Test_PHX_Task3_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_PHX_CheckGPTArch_Task3_Prompt123/Best_PHX_5000_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Feature Clean-up

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/
shopt -s extglob
rm -rf embeddings/!(README.md)

## HSS - 5000

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python features.py HSS 1 1 --gpt 0 --number 5000
python features.py HSS 2 1 --gpt 0 --number 5000

python features.py HSS 1 1 --gpt 1 --number 5000
python features.py HSS 1 2 --gpt 1 --number 5000
python features.py HSS 1 3 --gpt 1 --number 5000
python features.py HSS 1 4 --gpt 1 --number 5000

python features.py HSS 2 1 --gpt 1 --number 5000
python features.py HSS 2 2 --gpt 1 --number 5000
python features.py HSS 2 3 --gpt 1 --number 5000
python features.py HSS 2 4 --gpt 1 --number 5000

python features.py HSS 3 1 --gpt 1 --number 5000
python features.py HSS 3 2 --gpt 1 --number 5000
python features.py HSS 3 3 --gpt 1 --number 5000
python features.py HSS 3 4 --gpt 1 --number 5000

In [ ]:
elapsed_time(start)
start = now()

### Table 7 & 8 - CheckGPT

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_5000 1 1 Train_HSS_CheckGPTArch_Task1_Prompt1 --batchsize 256
python dnn.py HSS_5000 1 2 Train_HSS_CheckGPTArch_Task1_Prompt2 --batchsize 256
python dnn.py HSS_5000 1 3 Train_HSS_CheckGPTArch_Task1_Prompt3 --batchsize 256
python dnn.py HSS_5000 1 4 Train_HSS_CheckGPTArch_Task1_Prompt4 --batchsize 256

python dnn.py HSS_5000 2 1 Train_HSS_CheckGPTArch_Task2_Prompt1 --batchsize 256
python dnn.py HSS_5000 2 2 Train_HSS_CheckGPTArch_Task2_Prompt2 --batchsize 256
python dnn.py HSS_5000 2 3 Train_HSS_CheckGPTArch_Task2_Prompt3 --batchsize 256
python dnn.py HSS_5000 2 4 Train_HSS_CheckGPTArch_Task2_Prompt4 --batchsize 256

python dnn.py HSS_5000 3 1 Train_HSS_CheckGPTArch_Task3_Prompt1 --batchsize 256
python dnn.py HSS_5000 3 2 Train_HSS_CheckGPTArch_Task3_Prompt2 --batchsize 256
python dnn.py HSS_5000 3 3 Train_HSS_CheckGPTArch_Task3_Prompt3 --batchsize 256
python dnn.py HSS_5000 3 4 Train_HSS_CheckGPTArch_Task3_Prompt4 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_5000 1 1 Train_HSS_CheckGPTArch_Task1_Prompt1_Test_HSS_Task1_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CheckGPTArch_Task1_Prompt1/Best_HSS_5000_Task1.pth
python dnn.py HSS_5000 1 2 Train_HSS_CheckGPTArch_Task1_Prompt2_Test_HSS_Task1_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CheckGPTArch_Task1_Prompt2/Best_HSS_5000_Task1.pth
python dnn.py HSS_5000 1 3 Train_HSS_CheckGPTArch_Task1_Prompt3_Test_HSS_Task1_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CheckGPTArch_Task1_Prompt3/Best_HSS_5000_Task1.pth
python dnn.py HSS_5000 1 4 Train_HSS_CheckGPTArch_Task1_Prompt4_Test_HSS_Task1_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CheckGPTArch_Task1_Prompt4/Best_HSS_5000_Task1.pth

python dnn.py HSS_5000 2 1 Train_HSS_CheckGPTArch_Task2_Prompt1_Test_HSS_Task2_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CheckGPTArch_Task2_Prompt1/Best_HSS_5000_Task2.pth
python dnn.py HSS_5000 2 2 Train_HSS_CheckGPTArch_Task2_Prompt2_Test_HSS_Task2_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CheckGPTArch_Task2_Prompt2/Best_HSS_5000_Task2.pth
python dnn.py HSS_5000 2 3 Train_HSS_CheckGPTArch_Task2_Prompt3_Test_HSS_Task2_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CheckGPTArch_Task2_Prompt3/Best_HSS_5000_Task2.pth
python dnn.py HSS_5000 2 4 Train_HSS_CheckGPTArch_Task2_Prompt4_Test_HSS_Task2_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CheckGPTArch_Task2_Prompt4/Best_HSS_5000_Task2.pth

python dnn.py HSS_5000 3 1 Train_HSS_CheckGPTArch_Task3_Prompt1_Test_HSS_Task3_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CheckGPTArch_Task3_Prompt1/Best_HSS_5000_Task3.pth
python dnn.py HSS_5000 3 2 Train_HSS_CheckGPTArch_Task3_Prompt2_Test_HSS_Task3_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CheckGPTArch_Task3_Prompt2/Best_HSS_5000_Task3.pth
python dnn.py HSS_5000 3 3 Train_HSS_CheckGPTArch_Task3_Prompt3_Test_HSS_Task3_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CheckGPTArch_Task3_Prompt3/Best_HSS_5000_Task3.pth
python dnn.py HSS_5000 3 4 Train_HSS_CheckGPTArch_Task3_Prompt4_Test_HSS_Task3_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CheckGPTArch_Task3_Prompt4/Best_HSS_5000_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 8 - RCH

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_5000 1 1 Train_HSS_RCH_Task1_Prompt1 --batchsize 256 --modelid 2
python dnn.py HSS_5000 1 2 Train_HSS_RCH_Task1_Prompt2 --batchsize 256 --modelid 2
python dnn.py HSS_5000 1 3 Train_HSS_RCH_Task1_Prompt3 --batchsize 256 --modelid 2
python dnn.py HSS_5000 1 4 Train_HSS_RCH_Task1_Prompt4 --batchsize 256 --modelid 2

python dnn.py HSS_5000 2 1 Train_HSS_RCH_Task2_Prompt1 --batchsize 256 --modelid 2
python dnn.py HSS_5000 2 2 Train_HSS_RCH_Task2_Prompt2 --batchsize 256 --modelid 2
python dnn.py HSS_5000 2 3 Train_HSS_RCH_Task2_Prompt3 --batchsize 256 --modelid 2
python dnn.py HSS_5000 2 4 Train_HSS_RCH_Task2_Prompt4 --batchsize 256 --modelid 2

python dnn.py HSS_5000 3 1 Train_HSS_RCH_Task3_Prompt1 --batchsize 256 --modelid 2
python dnn.py HSS_5000 3 2 Train_HSS_RCH_Task3_Prompt2 --batchsize 256 --modelid 2
python dnn.py HSS_5000 3 3 Train_HSS_RCH_Task3_Prompt3 --batchsize 256 --modelid 2
python dnn.py HSS_5000 3 4 Train_HSS_RCH_Task3_Prompt4 --batchsize 256 --modelid 2

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_5000 1 1 Train_HSS_RCH_Task1_Prompt1_Test_HSS_Task1_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_RCH_Task1_Prompt1/Best_HSS_5000_Task1.pth
python dnn.py HSS_5000 1 2 Train_HSS_RCH_Task1_Prompt2_Test_HSS_Task1_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_RCH_Task1_Prompt2/Best_HSS_5000_Task1.pth
python dnn.py HSS_5000 1 3 Train_HSS_RCH_Task1_Prompt3_Test_HSS_Task1_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_RCH_Task1_Prompt3/Best_HSS_5000_Task1.pth
python dnn.py HSS_5000 1 4 Train_HSS_RCH_Task1_Prompt4_Test_HSS_Task1_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_RCH_Task1_Prompt4/Best_HSS_5000_Task1.pth

python dnn.py HSS_5000 2 1 Train_HSS_RCH_Task2_Prompt1_Test_HSS_Task2_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_RCH_Task2_Prompt1/Best_HSS_5000_Task2.pth
python dnn.py HSS_5000 2 2 Train_HSS_RCH_Task2_Prompt2_Test_HSS_Task2_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_RCH_Task2_Prompt2/Best_HSS_5000_Task2.pth
python dnn.py HSS_5000 2 3 Train_HSS_RCH_Task2_Prompt3_Test_HSS_Task2_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_RCH_Task2_Prompt3/Best_HSS_5000_Task2.pth
python dnn.py HSS_5000 2 4 Train_HSS_RCH_Task2_Prompt4_Test_HSS_Task2_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_RCH_Task2_Prompt4/Best_HSS_5000_Task2.pth

python dnn.py HSS_5000 3 1 Train_HSS_RCH_Task3_Prompt1_Test_HSS_Task3_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_RCH_Task3_Prompt1/Best_HSS_5000_Task3.pth
python dnn.py HSS_5000 3 2 Train_HSS_RCH_Task3_Prompt2_Test_HSS_Task3_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_RCH_Task3_Prompt2/Best_HSS_5000_Task3.pth
python dnn.py HSS_5000 3 3 Train_HSS_RCH_Task3_Prompt3_Test_HSS_Task3_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_RCH_Task3_Prompt3/Best_HSS_5000_Task3.pth
python dnn.py HSS_5000 3 4 Train_HSS_RCH_Task3_Prompt4_Test_HSS_Task3_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_RCH_Task3_Prompt4/Best_HSS_5000_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 8 - MLP

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_5000 1 1 Train_HSS_MLP_Task1_Prompt1 --batchsize 256 --modelid 3
python dnn.py HSS_5000 1 2 Train_HSS_MLP_Task1_Prompt2 --batchsize 256 --modelid 3
python dnn.py HSS_5000 1 3 Train_HSS_MLP_Task1_Prompt3 --batchsize 256 --modelid 3
python dnn.py HSS_5000 1 4 Train_HSS_MLP_Task1_Prompt4 --batchsize 256 --modelid 3

python dnn.py HSS_5000 2 1 Train_HSS_MLP_Task2_Prompt1 --batchsize 256 --modelid 3
python dnn.py HSS_5000 2 2 Train_HSS_MLP_Task2_Prompt2 --batchsize 256 --modelid 3
python dnn.py HSS_5000 2 3 Train_HSS_MLP_Task2_Prompt3 --batchsize 256 --modelid 3
python dnn.py HSS_5000 2 4 Train_HSS_MLP_Task2_Prompt4 --batchsize 256 --modelid 3

python dnn.py HSS_5000 3 1 Train_HSS_MLP_Task3_Prompt1 --batchsize 256 --modelid 3
python dnn.py HSS_5000 3 2 Train_HSS_MLP_Task3_Prompt2 --batchsize 256 --modelid 3
python dnn.py HSS_5000 3 3 Train_HSS_MLP_Task3_Prompt3 --batchsize 256 --modelid 3
python dnn.py HSS_5000 3 4 Train_HSS_MLP_Task3_Prompt4 --batchsize 256 --modelid 3

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_5000 1 1 Train_HSS_MLP_Task1_Prompt1_Test_HSS_Task1_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_MLP_Task1_Prompt1/Best_HSS_5000_Task1.pth
python dnn.py HSS_5000 1 2 Train_HSS_MLP_Task1_Prompt2_Test_HSS_Task1_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_MLP_Task1_Prompt2/Best_HSS_5000_Task1.pth
python dnn.py HSS_5000 1 3 Train_HSS_MLP_Task1_Prompt3_Test_HSS_Task1_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_MLP_Task1_Prompt3/Best_HSS_5000_Task1.pth
python dnn.py HSS_5000 1 4 Train_HSS_MLP_Task1_Prompt4_Test_HSS_Task1_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_MLP_Task1_Prompt4/Best_HSS_5000_Task1.pth

python dnn.py HSS_5000 2 1 Train_HSS_MLP_Task2_Prompt1_Test_HSS_Task2_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_MLP_Task2_Prompt1/Best_HSS_5000_Task2.pth
python dnn.py HSS_5000 2 2 Train_HSS_MLP_Task2_Prompt2_Test_HSS_Task2_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_MLP_Task2_Prompt2/Best_HSS_5000_Task2.pth
python dnn.py HSS_5000 2 3 Train_HSS_MLP_Task2_Prompt3_Test_HSS_Task2_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_MLP_Task2_Prompt3/Best_HSS_5000_Task2.pth
python dnn.py HSS_5000 2 4 Train_HSS_MLP_Task2_Prompt4_Test_HSS_Task2_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_MLP_Task2_Prompt4/Best_HSS_5000_Task2.pth

python dnn.py HSS_5000 3 1 Train_HSS_MLP_Task3_Prompt1_Test_HSS_Task3_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_MLP_Task3_Prompt1/Best_HSS_5000_Task3.pth
python dnn.py HSS_5000 3 2 Train_HSS_MLP_Task3_Prompt2_Test_HSS_Task3_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_MLP_Task3_Prompt2/Best_HSS_5000_Task3.pth
python dnn.py HSS_5000 3 3 Train_HSS_MLP_Task3_Prompt3_Test_HSS_Task3_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_MLP_Task3_Prompt3/Best_HSS_5000_Task3.pth
python dnn.py HSS_5000 3 4 Train_HSS_MLP_Task3_Prompt4_Test_HSS_Task3_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_MLP_Task3_Prompt4/Best_HSS_5000_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 8 - CNN

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_5000 1 1 Train_HSS_CNN_Task1_Prompt1 --batchsize 256 --modelid 4
python dnn.py HSS_5000 1 2 Train_HSS_CNN_Task1_Prompt2 --batchsize 256 --modelid 4
python dnn.py HSS_5000 1 3 Train_HSS_CNN_Task1_Prompt3 --batchsize 256 --modelid 4
python dnn.py HSS_5000 1 4 Train_HSS_CNN_Task1_Prompt4 --batchsize 256 --modelid 4

python dnn.py HSS_5000 2 1 Train_HSS_CNN_Task2_Prompt1 --batchsize 256 --modelid 4
python dnn.py HSS_5000 2 2 Train_HSS_CNN_Task2_Prompt2 --batchsize 256 --modelid 4
python dnn.py HSS_5000 2 3 Train_HSS_CNN_Task2_Prompt3 --batchsize 256 --modelid 4
python dnn.py HSS_5000 2 4 Train_HSS_CNN_Task2_Prompt4 --batchsize 256 --modelid 4

python dnn.py HSS_5000 3 1 Train_HSS_CNN_Task3_Prompt1 --batchsize 256 --modelid 4
python dnn.py HSS_5000 3 2 Train_HSS_CNN_Task3_Prompt2 --batchsize 256 --modelid 4
python dnn.py HSS_5000 3 3 Train_HSS_CNN_Task3_Prompt3 --batchsize 256 --modelid 4
python dnn.py HSS_5000 3 4 Train_HSS_CNN_Task3_Prompt4 --batchsize 256 --modelid 4

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_5000 1 1 Train_HSS_CNN_Task1_Prompt1_Test_HSS_Task1_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CNN_Task1_Prompt1/Best_HSS_5000_Task1.pth
python dnn.py HSS_5000 1 2 Train_HSS_CNN_Task1_Prompt2_Test_HSS_Task1_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CNN_Task1_Prompt2/Best_HSS_5000_Task1.pth
python dnn.py HSS_5000 1 3 Train_HSS_CNN_Task1_Prompt3_Test_HSS_Task1_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CNN_Task1_Prompt3/Best_HSS_5000_Task1.pth
python dnn.py HSS_5000 1 4 Train_HSS_CNN_Task1_Prompt4_Test_HSS_Task1_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CNN_Task1_Prompt4/Best_HSS_5000_Task1.pth

python dnn.py HSS_5000 2 1 Train_HSS_CNN_Task2_Prompt1_Test_HSS_Task2_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CNN_Task2_Prompt1/Best_HSS_5000_Task2.pth
python dnn.py HSS_5000 2 2 Train_HSS_CNN_Task2_Prompt2_Test_HSS_Task2_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CNN_Task2_Prompt2/Best_HSS_5000_Task2.pth
python dnn.py HSS_5000 2 3 Train_HSS_CNN_Task2_Prompt3_Test_HSS_Task2_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CNN_Task2_Prompt3/Best_HSS_5000_Task2.pth
python dnn.py HSS_5000 2 4 Train_HSS_CNN_Task2_Prompt4_Test_HSS_Task2_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CNN_Task2_Prompt4/Best_HSS_5000_Task2.pth

python dnn.py HSS_5000 3 1 Train_HSS_CNN_Task3_Prompt1_Test_HSS_Task3_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CNN_Task3_Prompt1/Best_HSS_5000_Task3.pth
python dnn.py HSS_5000 3 2 Train_HSS_CNN_Task3_Prompt2_Test_HSS_Task3_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CNN_Task3_Prompt2/Best_HSS_5000_Task3.pth
python dnn.py HSS_5000 3 3 Train_HSS_CNN_Task3_Prompt3_Test_HSS_Task3_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CNN_Task3_Prompt3/Best_HSS_5000_Task3.pth
python dnn.py HSS_5000 3 4 Train_HSS_CNN_Task3_Prompt4_Test_HSS_Task3_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CNN_Task3_Prompt4/Best_HSS_5000_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 8 - BiLSTM w/o attention

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_5000 1 1 Train_HSS_BiLSTMwoAttention_Task1_Prompt1 --batchsize 256 --modelid 1
python dnn.py HSS_5000 1 2 Train_HSS_BiLSTMwoAttention_Task1_Prompt2 --batchsize 256 --modelid 1
python dnn.py HSS_5000 1 3 Train_HSS_BiLSTMwoAttention_Task1_Prompt3 --batchsize 256 --modelid 1
python dnn.py HSS_5000 1 4 Train_HSS_BiLSTMwoAttention_Task1_Prompt4 --batchsize 256 --modelid 1

python dnn.py HSS_5000 2 1 Train_HSS_BiLSTMwoAttention_Task2_Prompt1 --batchsize 256 --modelid 1
python dnn.py HSS_5000 2 2 Train_HSS_BiLSTMwoAttention_Task2_Prompt2 --batchsize 256 --modelid 1
python dnn.py HSS_5000 2 3 Train_HSS_BiLSTMwoAttention_Task2_Prompt3 --batchsize 256 --modelid 1
python dnn.py HSS_5000 2 4 Train_HSS_BiLSTMwoAttention_Task2_Prompt4 --batchsize 256 --modelid 1

python dnn.py HSS_5000 3 1 Train_HSS_BiLSTMwoAttention_Task3_Prompt1 --batchsize 256 --modelid 1
python dnn.py HSS_5000 3 2 Train_HSS_BiLSTMwoAttention_Task3_Prompt2 --batchsize 256 --modelid 1
python dnn.py HSS_5000 3 3 Train_HSS_BiLSTMwoAttention_Task3_Prompt3 --batchsize 256 --modelid 1
python dnn.py HSS_5000 3 4 Train_HSS_BiLSTMwoAttention_Task3_Prompt4 --batchsize 256 --modelid 1

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_5000 1 1 Train_HSS_BiLSTMwoAttention_Task1_Prompt1_Test_HSS_Task1_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_BiLSTMwoAttention_Task1_Prompt1/Best_HSS_5000_Task1.pth
python dnn.py HSS_5000 1 2 Train_HSS_BiLSTMwoAttention_Task1_Prompt2_Test_HSS_Task1_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_BiLSTMwoAttention_Task1_Prompt2/Best_HSS_5000_Task1.pth
python dnn.py HSS_5000 1 3 Train_HSS_BiLSTMwoAttention_Task1_Prompt3_Test_HSS_Task1_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_BiLSTMwoAttention_Task1_Prompt3/Best_HSS_5000_Task1.pth
python dnn.py HSS_5000 1 4 Train_HSS_BiLSTMwoAttention_Task1_Prompt4_Test_HSS_Task1_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_BiLSTMwoAttention_Task1_Prompt4/Best_HSS_5000_Task1.pth

python dnn.py HSS_5000 2 1 Train_HSS_BiLSTMwoAttention_Task2_Prompt1_Test_HSS_Task2_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_BiLSTMwoAttention_Task2_Prompt1/Best_HSS_5000_Task2.pth
python dnn.py HSS_5000 2 2 Train_HSS_BiLSTMwoAttention_Task2_Prompt2_Test_HSS_Task2_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_BiLSTMwoAttention_Task2_Prompt2/Best_HSS_5000_Task2.pth
python dnn.py HSS_5000 2 3 Train_HSS_BiLSTMwoAttention_Task2_Prompt3_Test_HSS_Task2_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_BiLSTMwoAttention_Task2_Prompt3/Best_HSS_5000_Task2.pth
python dnn.py HSS_5000 2 4 Train_HSS_BiLSTMwoAttention_Task2_Prompt4_Test_HSS_Task2_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_BiLSTMwoAttention_Task2_Prompt4/Best_HSS_5000_Task2.pth

python dnn.py HSS_5000 3 1 Train_HSS_BiLSTMwoAttention_Task3_Prompt1_Test_HSS_Task3_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_BiLSTMwoAttention_Task3_Prompt1/Best_HSS_5000_Task3.pth
python dnn.py HSS_5000 3 2 Train_HSS_BiLSTMwoAttention_Task3_Prompt2_Test_HSS_Task3_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_BiLSTMwoAttention_Task3_Prompt2/Best_HSS_5000_Task3.pth
python dnn.py HSS_5000 3 3 Train_HSS_BiLSTMwoAttention_Task3_Prompt3_Test_HSS_Task3_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_BiLSTMwoAttention_Task3_Prompt3/Best_HSS_5000_Task3.pth
python dnn.py HSS_5000 3 4 Train_HSS_BiLSTMwoAttention_Task3_Prompt4_Test_HSS_Task3_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_BiLSTMwoAttention_Task3_Prompt4/Best_HSS_5000_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 9 - Cross Prompt

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_5000 1 0 Train_HSS_CheckGPTArch_Task1_Prompt1234_Test_HSS_Task1_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/HSS_Task1_Prompt1234/Best_LIT_Task1.pth
python dnn.py HSS_5000 2 0 Train_HSS_CheckGPTArch_Task2_Prompt1234_Test_HSS_Task2_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/HSS_Task2_Prompt1234/Best_LIT_Task2.pth
python dnn.py HSS_5000 3 0 Train_HSS_CheckGPTArch_Task3_Prompt1234_Test_HSS_Task3_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/HSS_Task3_Prompt1234/Best_LIT_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 9 - Cross Prompt,Domain

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_5000 1 0 Train_CSPHXHSS_CheckGPTArch_Task1_Prompt1234_Test_HSS_Task1_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/ALL_Task1_Prompt1234/Best_ALL_Task1.pth
python dnn.py HSS_5000 2 0 Train_CSPHXHSS_CheckGPTArch_Task2_Prompt1234_Test_HSS_Task2_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/ALL_Task2_Prompt1234/Best_ALL_Task2.pth
python dnn.py HSS_5000 3 0 Train_CSPHXHSS_CheckGPTArch_Task3_Prompt1234_Test_HSS_Task3_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/ALL_Task3_Prompt1234/Best_ALL_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 9 - Cross Prompt,Domain,Task

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_5000 1 0 Train_CSPHXHSS_CheckGPTArch_Task123_Prompt1234_Test_HSS_Task1_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/ALL_Task123_Prompt1234/Best_ALL_Task0.pth
python dnn.py HSS_5000 2 0 Train_CSPHXHSS_CheckGPTArch_Task123_Prompt1234_Test_HSS_Task2_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/ALL_Task123_Prompt1234/Best_ALL_Task0.pth
python dnn.py HSS_5000 3 0 Train_CSPHXHSS_CheckGPTArch_Task123_Prompt1234_Test_HSS_Task3_Prompt1234 --pretrain 1 --test 1 --saved-model ../CheckGPT_presaved_files/saved_experiments/basic/ALL_Task123_Prompt1234/Best_ALL_Task0.pth

### Table 10 - Prompt 234

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_5000 1 234 Train_HSS_CheckGPTArch_Task1_Prompt234 --batchsize 256
python dnn.py HSS_5000 2 234 Train_HSS_CheckGPTArch_Task2_Prompt234 --batchsize 256
python dnn.py HSS_5000 3 234 Train_HSS_CheckGPTArch_Task3_Prompt234 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_5000 1 1 Train_HSS_CheckGPTArch_Task1_Prompt234_Test_HSS_Task1_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CheckGPTArch_Task1_Prompt234/Best_HSS_5000_Task1.pth
python dnn.py HSS_5000 2 1 Train_HSS_CheckGPTArch_Task2_Prompt234_Test_HSS_Task2_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CheckGPTArch_Task2_Prompt234/Best_HSS_5000_Task2.pth
python dnn.py HSS_5000 3 1 Train_HSS_CheckGPTArch_Task3_Prompt234_Test_HSS_Task3_Prompt1 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CheckGPTArch_Task3_Prompt234/Best_HSS_5000_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 10 - Prompt 134

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_5000 1 134 Train_HSS_CheckGPTArch_Task1_Prompt134 --batchsize 256
python dnn.py HSS_5000 2 134 Train_HSS_CheckGPTArch_Task2_Prompt134 --batchsize 256
python dnn.py HSS_5000 3 134 Train_HSS_CheckGPTArch_Task3_Prompt134 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_5000 1 2 Train_HSS_CheckGPTArch_Task1_Prompt134_Test_HSS_Task1_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CheckGPTArch_Task1_Prompt134/Best_HSS_5000_Task1.pth
python dnn.py HSS_5000 2 2 Train_HSS_CheckGPTArch_Task2_Prompt134_Test_HSS_Task2_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CheckGPTArch_Task2_Prompt134/Best_HSS_5000_Task2.pth
python dnn.py HSS_5000 3 2 Train_HSS_CheckGPTArch_Task3_Prompt134_Test_HSS_Task3_Prompt2 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CheckGPTArch_Task3_Prompt134/Best_HSS_5000_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 10 - Prompt 124

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_5000 1 124 Train_HSS_CheckGPTArch_Task1_Prompt124 --batchsize 256
python dnn.py HSS_5000 2 124 Train_HSS_CheckGPTArch_Task2_Prompt124 --batchsize 256
python dnn.py HSS_5000 3 124 Train_HSS_CheckGPTArch_Task3_Prompt124 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_5000 1 3 Train_HSS_CheckGPTArch_Task1_Prompt124_Test_HSS_Task1_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CheckGPTArch_Task1_Prompt124/Best_HSS_5000_Task1.pth
python dnn.py HSS_5000 2 3 Train_HSS_CheckGPTArch_Task2_Prompt124_Test_HSS_Task2_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CheckGPTArch_Task2_Prompt124/Best_HSS_5000_Task2.pth
python dnn.py HSS_5000 3 3 Train_HSS_CheckGPTArch_Task3_Prompt124_Test_HSS_Task3_Prompt3 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CheckGPTArch_Task3_Prompt124/Best_HSS_5000_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Table 10 - Prompt 123

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_5000 1 123 Train_HSS_CheckGPTArch_Task1_Prompt123 --batchsize 256
python dnn.py HSS_5000 2 123 Train_HSS_CheckGPTArch_Task2_Prompt123 --batchsize 256
python dnn.py HSS_5000 3 123 Train_HSS_CheckGPTArch_Task3_Prompt123 --batchsize 256

In [ ]:
elapsed_time(start)
start = now()

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/

python dnn.py HSS_5000 1 4 Train_HSS_CheckGPTArch_Task1_Prompt123_Test_HSS_Task1_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CheckGPTArch_Task1_Prompt123/Best_HSS_5000_Task1.pth
python dnn.py HSS_5000 2 4 Train_HSS_CheckGPTArch_Task2_Prompt123_Test_HSS_Task2_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CheckGPTArch_Task2_Prompt123/Best_HSS_5000_Task2.pth
python dnn.py HSS_5000 3 4 Train_HSS_CheckGPTArch_Task3_Prompt123_Test_HSS_Task3_Prompt4 --pretrain 1 --test 1 --saved-model ./exp/Train_HSS_CheckGPTArch_Task3_Prompt123/Best_HSS_5000_Task3.pth

In [ ]:
elapsed_time(start)
start = now()

### Feature Clean-up

In [ ]:
%%bash
source ./artifact_checkgpt/.venv/bin/activate
cd ~/CheckGPT-reproduction/artifact_checkgpt/CheckGPT/
shopt -s extglob
rm -rf embeddings/!(README.md)

In [ ]:
elapsed_time(start)
elapsed_time(global_start)